Business Intelligence

Time Series Analysis
- Are there monthly and weekly sales seasonality?
- Are there anomalous* peaks and troughs - what could explain them?
- How does every year from 2015-2019 compare against each other?
  
Distribution and Aggregations
- What's the distribution of sales per product category^? 
- What's the dollar value per month/year per:
  - individual
  - cluster (once identified)
  - product category

In [2]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [94]:
# member = pd.read_csv("cl_member.csv")
guest = pd.read_csv("cl_guest.csv")

In [226]:
# df = member[["acn","timestamp","item","item_group","total","sys_id"]]
# df = df.rename(columns={"acn":"id","sys_id":"product_category"})

In [95]:
guest.head()

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
0,0,201066,2019-03-29 20:42:00,Lara Cornell,24 Hours of Tennis Ping Pong,24 Hour Charity Event,Paid,2.0,20.0,0.0,0.00,20.00,20.00,0.0,0.0,NaN,4283
1,1,152150,2018-04-17 18:26:00,Melanie Bishop,Kick Off Clinic,24 Hour Charity Event,Paid,1.0,25.0,0.0,0.00,25.00,25.00,0.0,0.0,NaN,1656
2,2,198364,2019-03-11 10:48:00,Frost Bank,Frost Bank- 24 Hours of Tennis Community Sponsor,24 Hour- Sponsorship,Paid,1.0,500.0,0.0,0.00,500.00,500.00,0.0,0.0,NaN,1864
3,3,204816,2019-04-22 11:06:00,Lisa Munekawa,JNCC HAT,Accessories,Paid,1.0,17.0,0.0,1.40,18.40,18.40,0.0,0.0,NaN,4629
4,4,16014,2015-03-11 18:29:00,Jennifer Growcock,Pro Overgrip,Accessories,Paid,1.0,8.0,0.0,0.66,8.66,8.66,0.0,0.0,NaN,1317


In [ ]:
Index(['Unnamed: 0', 'trans_id', 'timestamp', 'acn', 'item', 'item_group',
       'total', 'sys_id'],
      dtype='object')

In [97]:
df2 = guest[['Unnamed: 0',"trans_id","timestamp","sys_id","item","item_group","total","acn"]]
df2 = df2.rename(columns={"sys_id":"acn","acn":"sys_id"})
df2["sys_id"] = "Guest"

In [98]:
df2.head()

,Unnamed: 0,trans_id,timestamp,acn,item,item_group,total,sys_id
0,0,201066,2019-03-29 20:42:00,4283,24 Hours of Tennis Ping Pong,24 Hour Charity Event,20.00,Guest
1,1,152150,2018-04-17 18:26:00,1656,Kick Off Clinic,24 Hour Charity Event,25.00,Guest
2,2,198364,2019-03-11 10:48:00,1864,Frost Bank- 24 Hours of Tennis Community Sponsor,24 Hour- Sponsorship,500.00,Guest
3,3,204816,2019-04-22 11:06:00,4629,JNCC HAT,Accessories,18.40,Guest
4,4,16014,2015-03-11 18:29:00,1317,Pro Overgrip,Accessories,8.66,Guest


**Task: Concat df and df2**  
But first, check if ids are unique, if not find out why not.

In [229]:
member.head()

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
0,0,7315,2015-01-03 11:00:00,Pia Lomax,Lessons with Joe Snailum (01/03/2015),Lessons,Paid,1.0,65.00,0.00,0.00,65.00,65.00,0.0,0.0,79,General Single Membership
1,1,37831,2015-08-12 21:30:00,Jennifer Hernandez,$1 Snacks,Snacks,Paid,1.0,0.92,0.00,0.08,1.00,1.00,0.0,0.0,566,General Family Membership
2,2,37844,2015-08-13 10:50:00,Douglas Draeger,Java Jolt,Smoothie,Paid,1.0,5.31,0.53,0.39,5.17,5.17,0.0,0.0,1532,General Property Owner
3,3,37843,2015-08-13 10:50:00,Douglas Draeger,Java Jolt,Smoothie,Paid,1.0,5.31,0.53,0.39,5.17,5.17,0.0,0.0,1532,General Property Owner
4,4,37842,2015-08-13 10:50:00,Douglas Draeger,Mango Tango,Smoothie,Paid,1.0,4.39,0.44,0.33,4.28,4.28,0.0,0.0,1532,General Property Owner


In [230]:
df[df.id.isin(df2.id)].id.count()

971

In [231]:
dups = df[df.id.isin(df2.id)].id.unique()

In [232]:
member[member.acn.isin(dups)].name.unique()

array(['Ashley Lloyd', 'Lee Stacy', 'Rhonda Stacy', 'Diego Garcia',
       'Hartley Clark', 'Evyn Clark', 'Abigail Kelly', 'Cole Jecmenek',
       'David Smith', 'Khloe Villa', 'Aaron Malakoff', 'Jessica Villa',
       'Ryan Villa'], dtype=object)

Check from the cancellation data.

Duplicated entries who are **confirmed** in the Cancellation data

- Aaron Malakoff  
- Rhonda, Lee Stacy  
- David Smith

Duplicated entries who have **similar names** in the Cancellation data
- Roberto same as Diego Garcia?  
- Wes same as Hartley/Evyn Clark?  
- Jarrett same as Cole Jecmenek?

Duplicated entries who are **not** in the Cancellation data
- Ashley Lloyd  
- Abigail Kelly
- Jessica Villa

In [233]:
member.item_group.unique().tolist()

['Lessons',
 'Snacks',
 'Smoothie',
 'White Wine',
 'Drinks',
 'Tips',
 'Guest Fee',
 'Apparel',
 'Accessories',
 'Tennis Balls',
 'Adult Clinics',
 'Massage',
 'Membership Initiation ',
 'Membership Dues',
 'Junior Clinics',
 'Beer',
 'Date Night',
 'Racquet Rental',
 'Red Wine',
 'String',
 'Package Liability',
 'Restring/ Labor Only',
 'Special Event',
 'Shoes',
 'Facility Rental',
 'Dinners',
 'Camp',
 'Racquets',
 'Leagues',
 'Personal Training',
 'Catie Johnson Home School',
 'Fitness Special Event',
 'Pro Shop',
 'Gift Certificate Liability',
 'Social Event',
 'Swim Lessons ',
 'Credits',
 'Swim Camp',
 'TRAIN4LIFE',
 'Mini Bees Soccer',
 'Teen Performance Camp',
 'CDM- MISC',
 'Soccer- MSOS',
 'Special Event-Non Taxable (Includes Instruction)',
 'Food Allowance',
 'Naturally Cafe',
 'Naturally-Salad',
 'Naturally Lunch Special',
 'Naturally Juices',
 'Naturally-Smoothies',
 'Naturally- Sandwiches',
 'Court Time',
 'VIP TENNIS',
 'VIP FITNESS',
 'Food & Beverage',
 'Kid Fit',
 '

### Extinct or exclude
- Massage**
- CDM-MISC
- Soccer-MSOS
- Membership Initiation Liability 
- Replacement Key Card
- TRAIN4LIFE - drop
- Mini Bees Soccer- drop
- Replacement Key Card - drop
- Catie Johnson Home School - drop
- VIP TENNIS - drop
- VIP FITNESS - drop

Rename Package Liability to Lessons

In [99]:
df2.head()

,Unnamed: 0,trans_id,timestamp,acn,item,item_group,total,sys_id
0,0,201066,2019-03-29 20:42:00,4283,24 Hours of Tennis Ping Pong,24 Hour Charity Event,20.00,Guest
1,1,152150,2018-04-17 18:26:00,1656,Kick Off Clinic,24 Hour Charity Event,25.00,Guest
2,2,198364,2019-03-11 10:48:00,1864,Frost Bank- 24 Hours of Tennis Community Sponsor,24 Hour- Sponsorship,500.00,Guest
3,3,204816,2019-04-22 11:06:00,4629,JNCC HAT,Accessories,18.40,Guest
4,4,16014,2015-03-11 18:29:00,1317,Pro Overgrip,Accessories,8.66,Guest


In [100]:
#Function for Exclusions and Package Liability
def exclusion_and_pl(member):
    #Exclusion
    member = member[~member.item_group.isin(["Massage","CDM- MISC","Soccer-MSOS","Membership Initiation Liability","Replacement Key Card","TRAIN4LIFE","Mini Bees Soccer","Catie Johnson Home School","VIP TENNIS","VIP FITNESS"])]

    #Regroup Package Liability
    package_drop = ['Added an additional lesson for Accidental Cancellation Charge- it was reduced from package- MD']
    package_fitness = ['PREMIER -1 hour- 4 pack (4 pack)', '1 hour - 4 pack (4 pack)','1 hour - 4 pack (4 pack) (10% discount applied)','REGULAR 1 hour - 4 pack (4 pack)','REGULAR 30 minute - 4 pack (4 pack)','1 Hour Lesson Package (4 pack)','30 Minute Lesson Package (4 pack)','1 Hour Lesson Package (4 pack) (10% discount applied)','Regular Lesson Package* mal (6 pack) (10% discount applied)','September 2017 Membership Promo (6 pack)','6 for 360 Fitness (6 pack)','3 for 99 FITNESS 2019-2020 (3 pack)','December 3- for $99.00 w/ Carly (3 pack)','Private Lesson 5 Pack - 1 on 1 (Kayla Ward)']
    package_tennis = ['LESSON 4 Pack - 1 on 1 (Tennis)','Premier 4 Pack - 1 on 1 (Tennis)','LESSON 8 Pack - 1 on 1 (Tennis)','Premier 8 Pack - 1 on 1 (Tennis)','Additional 3 Sessions for LESSON 4 Pack - 1 on 1 (Cal Loveridge)  (8575)','LESSON 4 Pack - 1 on 1 (Cal Loveridge)','LESSON 6 Pack 1 on 1 (Tennis)','December- 3 for $99 w/ Sam Cassidy (3 pack)','Tyler Brown $99.00 (3 pack)', '6 for 360 Tennis (6 pack)','3 FOR 99 TENNIS 2019-2020 (3 pack)','LESSON 3 Pack 1 on 1 (Teg Lozano)','NBCA STUDENT LESSON (3 pack)','Private Lesson 3 Pack 1 on 1 (Tennis) PTU Pro Package']

    mask_pl = member.item_group=="Package Liability"

    mask_package_drop = member.item.isin(package_drop)
    mask_package_fitness = member.item.isin(package_fitness)
    mask_package_tennis = member.item.isin(package_tennis)

    member.item_group = np.where(mask_package_fitness,"Fitness Lessons",member.item_group)
    member.item_group = np.where(mask_package_tennis,"Tennis Lessons",member.item_group)

    member.drop((member[member.item.isin(package_drop)].index),inplace=True)
    
    return member

In [101]:
df2 = exclusion_and_pl(df2)
df2.head()

,Unnamed: 0,trans_id,timestamp,acn,item,item_group,total,sys_id
0,0,201066,2019-03-29 20:42:00,4283,24 Hours of Tennis Ping Pong,24 Hour Charity Event,20.00,Guest
1,1,152150,2018-04-17 18:26:00,1656,Kick Off Clinic,24 Hour Charity Event,25.00,Guest
2,2,198364,2019-03-11 10:48:00,1864,Frost Bank- 24 Hours of Tennis Community Sponsor,24 Hour- Sponsorship,500.00,Guest
3,3,204816,2019-04-22 11:06:00,4629,JNCC HAT,Accessories,18.40,Guest
4,4,16014,2015-03-11 18:29:00,1317,Pro Overgrip,Accessories,8.66,Guest


In [102]:
member = member[~member.item_group.isin(["Massage","CDM- MISC","Soccer-MSOS","Membership Initiation Liability","Replacement Key Card","TRAIN4LIFE","Mini Bees Soccer","Catie Johnson Home School","VIP TENNIS","VIP FITNESS"])]

In [103]:
#1
package_drop = ['Added an additional lesson for Accidental Cancellation Charge- it was reduced from package- MD']
#565
package_fitness = ['PREMIER -1 hour- 4 pack (4 pack)', '1 hour - 4 pack (4 pack)','1 hour - 4 pack (4 pack) (10% discount applied)','REGULAR 1 hour - 4 pack (4 pack)','REGULAR 30 minute - 4 pack (4 pack)','1 Hour Lesson Package (4 pack)','30 Minute Lesson Package (4 pack)','1 Hour Lesson Package (4 pack) (10% discount applied)','Regular Lesson Package* mal (6 pack) (10% discount applied)','September 2017 Membership Promo (6 pack)','6 for 360 Fitness (6 pack)','3 for 99 FITNESS 2019-2020 (3 pack)','December 3- for $99.00 w/ Carly (3 pack)','Private Lesson 5 Pack - 1 on 1 (Kayla Ward)']
#241
package_tennis = ['LESSON 4 Pack - 1 on 1 (Tennis)','Premier 4 Pack - 1 on 1 (Tennis)','LESSON 8 Pack - 1 on 1 (Tennis)','Premier 8 Pack - 1 on 1 (Tennis)','Additional 3 Sessions for LESSON 4 Pack - 1 on 1 (Cal Loveridge)  (8575)','LESSON 4 Pack - 1 on 1 (Cal Loveridge)','LESSON 6 Pack 1 on 1 (Tennis)','December- 3 for $99 w/ Sam Cassidy (3 pack)','Tyler Brown $99.00 (3 pack)', '6 for 360 Tennis (6 pack)','3 FOR 99 TENNIS 2019-2020 (3 pack)','LESSON 3 Pack 1 on 1 (Teg Lozano)','NBCA STUDENT LESSON (3 pack)','Private Lesson 3 Pack 1 on 1 (Tennis) PTU Pro Package']

In [104]:
mask_pl = member.item_group=="Package Liability"

In [105]:
mask_package_drop = member.item.isin(package_drop)
mask_package_fitness = member.item.isin(package_fitness)
mask_package_tennis = member.item.isin(package_tennis)

In [106]:
member.item_group = np.where(mask_package_fitness,"Fitness Lessons",member.item_group)

In [107]:
df2.item_group.unique()

array(['24 Hour Charity Event', '24 Hour- Sponsorship', 'Accessories',
       'Adult Clinics', 'Apparel', 'Beer', 'Camp', 'Date Night',
       'Dinners', 'Drinks', 'Facility Rental', 'Fitness Special Event',
       'Food & Beverage', 'Food Allowance', 'Gift Certificate Liability',
       'Guest Fee', 'Junior Clinics', 'Kid Fit', 'Leagues', 'Lessons',
       'Membership Dues', 'Membership Initiation ', 'Naturally Cafe',
       'Naturally Juices', 'Naturally Lunch Special',
       'Naturally- Sandwiches', 'Naturally-Salad', 'Naturally-Smoothies',
       'Package Liability', 'Tennis Lessons', 'Personal Training',
       'Racquet Rental', 'Racquets', 'Red Wine', 'Restring/ Labor Only',
       'Shoes', 'Smoothie', 'Snacks', 'Social Event', 'Special Event',
       'Sponsorship', 'String', 'Swim Camp', 'Teen Performance Camp',
       'Tennis Balls', 'Tips', 'White Wine'], dtype=object)

In [108]:
df2[df2.item_group == "Package Liability"]

,Unnamed: 0,trans_id,timestamp,acn,item,item_group,total,sys_id
4714,4714,222728,2019-08-07 15:44:00,4949,Private Lesson 8 Pack 1 on 1 (Fitness),Package Liability,520.0,Guest
4715,4715,48484,2015-11-05 17:31:00,1690,Non Member 30 Minute- 4 Pack (4 pack),Package Liability,130.0,Guest


In [109]:
df2.loc[4715,"item_group"] = "Tennis Lessons"

In [110]:
df2.loc[4714,"item_group"] = "Fitness Lessons"

In [111]:
df2.loc[4714]

Unnamed: 0                                      4714
trans_id                                      222728
timestamp                        2019-08-07 15:44:00
acn                                             4949
item          Private Lesson 8 Pack 1 on 1 (Fitness)
item_group                           Fitness Lessons
total                                            520
sys_id                                         Guest
Name: 4714, dtype: object

In [112]:
#checking if Tennis Lessons are set as item_group
member.iloc[200]

Unnamed: 0                    206
trans_id                    37626
timestamp     2015-08-10 12:47:00
acn                           863
item              $2.00 Ice Cream
item_group                 Snacks
total                           2
sys_id           Misc. Membership
Name: 200, dtype: object

In [113]:
member.item_group = np.where(mask_package_tennis,"Tennis Lessons",member.item_group)

In [114]:
member.drop((member[member.item.isin(package_drop)].index),inplace=True)

In [115]:
member.shape

(135822, 8)

---

In [116]:
sorted(list(df2.item_group.unique()))

['24 Hour Charity Event',
 '24 Hour- Sponsorship',
 'Accessories',
 'Adult Clinics',
 'Apparel',
 'Beer',
 'Camp',
 'Date Night',
 'Dinners',
 'Drinks',
 'Facility Rental',
 'Fitness Lessons',
 'Fitness Special Event',
 'Food & Beverage',
 'Food Allowance',
 'Gift Certificate Liability',
 'Guest Fee',
 'Junior Clinics',
 'Kid Fit',
 'Leagues',
 'Lessons',
 'Membership Dues',
 'Membership Initiation ',
 'Naturally Cafe',
 'Naturally Juices',
 'Naturally Lunch Special',
 'Naturally- Sandwiches',
 'Naturally-Salad',
 'Naturally-Smoothies',
 'Personal Training',
 'Racquet Rental',
 'Racquets',
 'Red Wine',
 'Restring/ Labor Only',
 'Shoes',
 'Smoothie',
 'Snacks',
 'Social Event',
 'Special Event',
 'Sponsorship',
 'String',
 'Swim Camp',
 'Teen Performance Camp',
 'Tennis Balls',
 'Tennis Lessons',
 'Tips',
 'White Wine']

In [117]:
df2 = df2[df2.item_group != "Gift Certificate Liability"]

In [118]:
df2[df2.item_group=="Food & Beverage"]

,Unnamed: 0,trans_id,timestamp,acn,item,item_group,total,sys_id
2043,2043,162754,2018-06-30 00:00:00,1652,Monthly Food and Beverage Allowance July 2018,Food & Beverage,21.65,Guest
2044,2044,168644,2018-07-31 00:00:00,1652,Monthly Food and Beverage Allowance August 2018,Food & Beverage,21.65,Guest
2045,2045,220540,2019-07-27 14:12:00,4838,50 Drink Tickets for Event!,Food & Beverage,324.75,Guest


In [119]:
df2.loc[2043,"item_group"] = "Food Allowance"
df2.loc[2044,"item_group"] = "Food Allowance"
df2.loc[2045,"item_group"] = "Beverage"

In [120]:
df2.loc[2045]

Unnamed: 0                           2045
trans_id                           220540
timestamp             2019-07-27 14:12:00
acn                                  4838
item          50 Drink Tickets for Event!
item_group                       Beverage
total                              324.75
sys_id                              Guest
Name: 2045, dtype: object

In [121]:
list(df2[df2.item_group =="Naturally Cafe"].item.unique())

['Slice of Quiche',
 'Soup Small',
 'NUUN Tablets',
 'Chips ONLY',
 'Cookie',
 '1/2 Soup 1/2 Quiche',
 'Honey Stingers - Protein Bars 1.5 oz $2.50',
 'Muffin',
 'Hot Chocolate',
 'Large Soup',
 'Grab and Go Box',
 'Honey Stingers - Organic Gels 1.1 oz $1.50',
 'Honey Stingers - Organic Waffle 1.0 oz $1.39',
 'Icelandic Yogurt',
 'Juice box',
 '1/2 Soup 1/2 Sandwich',
 'Honey Stingers - Organic Energy Chews 1.8 oz $2.39',
 'Kombucha']

In [122]:
nc_food = ['Slice of Quiche','Soup Small','NUUN Tablets','Chips ONLY','Cookie','1/2 Soup 1/2 Quiche','Honey Stingers - Protein Bars 1.5 oz $2.50','Muffin','Large Soup','Grab and Go Box','Honey Stingers - Organic Gels 1.1 oz $1.50','Honey Stingers - Organic Waffle 1.0 oz $1.39','1/2 Soup 1/2 Sandwich','Honey Stingers - Organic Energy Chews 1.8 oz $2.39','Kombucha']
nc_beverage = ['Hot Chocolate','Icelandic Yogurt','Juice box','Kombucha']

In [123]:
df2.item_group = np.where(df2.item.isin(nc_food),"Food & Consumables",df2.item_group)
df2.item_group = np.where(df2.item.isin(nc_beverage),"Beverage",df2.item_group)

In [124]:
df2.item_group = np.where(df2.item_group.isin(["Naturally-Salad","Naturally Lunch Special","Naturally- Sandwiches"]), "Food & Consumables", df2.item_group)
df2.item_group = np.where(df2.item_group.isin(["Naturally Juices","Naturally-Smoothies","Smoothie"]), "Beverage", df2.item_group)

In [125]:
df2 = df2[df2.item_group != "Tips"]

In [126]:
df2.item_group = np.where(df2.item_group == "Drinks","Beverage",df2.item_group)

In [127]:
df2.item_group = np.where(df2.item_group.isin(["Red Wine","White Wine"]),"Wine",df2.item_group)

In [128]:
df2.item_group = np.where(df2.item_group.isin(['24 Hour Charity Event','24 Hour- Sponsorship']),"24 Hour of Tennis",df2.item_group)

In [129]:
df2.shape

(6124, 8)

In [130]:
df2.to_csv("df_guest.csv")

---

In [243]:
member = member[member.item_group != "Gift Certificate Liability"]

In [244]:
nc_food = ['Chips ONLY','Chips ONLY(Stacey)','Slice of Quiche','Muffin','Grab and Go Box','Parfait cup 16 oz','Carrot Cake','Grab and Go Box (Vicki)','Cookie','Fruit cup 16 oz','Chips ONLY (Danika)','Boiled Egg (Emillie)','Banana','Chips ONLY(Mark)','Apple','1/2 Soup 1/2 Sandwich','Soup Small','Salad dressing','1/2 Soup 1/2 Quiche','Soup Small (Krista)','Cookie (Lendon)','Cookie (Alison)','Muffin ( Sandy)','Slice of Quiche (Dixie)','Large Soup','Whole Quiche','Chips ONLY (Sandy)','Muffin (Audra)','Chips ONLY (Ava)','Muffin (Ava)','Cookie (October Purchase)','Naturally Soup','Chicken Salad','Hard Boiled Egg','Chicken Add ons','Side of Chicken Salad','Kombucha','Honey Stingers - Protein Bars 1.5 oz $2.50','Meat & Cheese Box','Small Tea','Honey Stingers - Organic Gels 1.1 oz $1.50','Honey Stingers - Organic Gels 1.1 oz $1.50 (Amy)','Honey Stingers - Organic Energy Chews 1.8 oz $2.39','Icelandic Yogurt','Meat & Cheese Box (raegan)','Muffin (Sandy)','1/2 Soup 1/2 Salad','8 oz House Dressing','house dressing','Green Sauce','Honey Stingers - Protein Bars 1.5 oz $2.50 (2 or 4 Boxes)','Honey Stingers - Protein Bars 1.5 oz $2.50 (2 or 4 boxes)','Honey Stingers - Organic Waffle 1.0 oz $1.39','Soup Small- Krista Hall','Honey Stingers - Organic Waffle 1.0 oz $1.39 ( Jennifer)','1/2 Soup 1/2 Sandwich (Krista)','Meat & Cheese Box (Kathryn)','Custom Meat and Cheese Platter','Honey Stingers - Protein Bars 1.5 oz $2.50 (Dixie)','Muffin (Ginger)','Chips ONLY (Lisa)','Kombucha (Tasha)','Muffin (Stephanie)','Slice of Quiche (Stephanie)','Muffin (Ginny)','Chips ONLY (Henry)','Muffin (Kendra)','1/2 Soup 1/2 Quiche (Kendra)','Muffin (Holly)','Honey Stingers - GLUTEN FREE Organic Waffles 1.0','NUUN Tablets','1/2 Soup 1/2 Sandwich (Holly)','Muffin ( Holly)','Whole Quiche (Ginny)','Single Hard-Boiled Egg','Slice of Quiche (Tasha)','Muffin (Caroline)','Chips ONLY (Jacqueline)','Muffin (renata)','Cinnamon Roll Large','cinnamon roll','Cinnamon rolls','Muffin (Rhonda)']
nc_beverage = ['Juice box','Hot Chocolate','Large Tea','Hot Chocolate (Sandy)','Juice box ( Rachel)']

In [245]:
nc_wildcard = ['Double Charged for Chicken and Bacon on 02/12/19- Issued Credit on 03/02 Backdated','Estates POA: Naturally Catering','Naturally Catered Event Pam Powell Party on April 10th','Party Platter for party on 10/17/2018']

In [246]:
member.item_group = np.where(member.item.isin(nc_food),"Food & Consumables",member.item_group)
member.item_group = np.where(member.item.isin(nc_beverage),"Beverage",member.item_group)

In [247]:
member = member[~member.item.isin(nc_wildcard)]

In [248]:
member = member[member.item_group != "JNCC 100% RETAIL- NOT CDM"]

In [249]:
member.item_group = np.where(member.item_group =="Special Event-Non Taxable (Includes Instruction)","Special Event",member.item_group)

In [250]:
member = member[member.item_group != "Pro Shop"]

In [251]:
member.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136982 entries, 0 to 137331
Data columns (total 17 columns):
Unnamed: 0    136982 non-null int64
trans_id      136982 non-null int64
timestamp     136982 non-null object
name          136982 non-null object
item          136982 non-null object
item_group    136982 non-null object
status        136982 non-null object
quantity      136982 non-null float64
subtotal      136982 non-null float64
discount      136982 non-null float64
tax           136982 non-null float64
total         136982 non-null float64
paid          136982 non-null float64
refund        136982 non-null float64
balance       136982 non-null float64
acn           136982 non-null int64
sys_id        136982 non-null object
dtypes: float64(8), int64(3), object(6)
memory usage: 18.8+ MB


In [252]:
sorted(member.item_group.unique())

['24 Hour Charity Event',
 '24 Hour- Dinner/ Lunch Entry',
 '24 Hour- Sponsorship',
 '24 Hour- Ticket Sales',
 'Accessories',
 'Adult Clinics',
 'Apparel',
 'Beer',
 'Beverage',
 'Camp',
 'Court Time',
 'Credits',
 'Date Night',
 'Dinners',
 'Drinks',
 'Facility Rental',
 'Fitness Lessons',
 'Fitness Special Event',
 'Food & Beverage',
 'Food & Consumables',
 'Food Allowance',
 'Guest Fee',
 'Homeowner Membership Initiation',
 'Junior Clinics',
 'Kid Fit',
 'Leagues',
 'Lessons',
 'Membership Dues',
 'Membership Initiation ',
 'Naturally Juices',
 'Naturally Lunch Special',
 'Naturally- Sandwiches',
 'Naturally-Salad',
 'Naturally-Smoothies',
 'Personal Training',
 'Racquet Rental',
 'Racquets',
 'Red Wine',
 'Restring/ Labor Only',
 'Shoes',
 'Smoothie',
 'Snacks',
 'Soccer- MSOS',
 'Social Event',
 'Special Event',
 'String',
 'Swim Camp',
 'Swim Lessons ',
 'Teen Performance Camp',
 'Tennis Balls',
 'Tennis Lessons',
 'Tips',
 'White Wine']

In [253]:
member.item_group = np.where(member.item_group.isin(["Naturally-Salad","Naturally Lunch Special","Naturally- Sandwiches"]), "Food & Consumables", member.item_group)
member.item_group = np.where(member.item_group.isin(["Naturally Juices","Naturally-Smoothies","Smoothie"]), "Beverage", member.item_group)

In [254]:
list(member[member.item_group == "Food & Beverage"].item.unique())

['6 bottles of Autin Hope 25$ per bottle',
 'Monthly Food and Beverage Allowance August 2018',
 'Monthly Food and Beverage Allowance July 2018',
 'Monthly Food and Beverage Allowance June 2018',
 'Austin Hope Red Wine',
 'Estates POA Party Wine Tasting Bottles']

In [255]:
member.item_group = np.where(member.item.isin(['Monthly Food and Beverage Allowance August 2018','Monthly Food and Beverage Allowance July 2018','Monthly Food and Beverage Allowance June 2018']), "Food Allowance", member.item_group)
member.item_group = np.where(member.item.isin(['Austin Hope Red Wine','Estates POA Party Wine Tasting Bottles','6 bottles of Autin Hope 25$ per bottle']), "Wine", member.item_group)
member.item_group = np.where(member.item_group.isin(["Red Wine","White Wine"]),"Wine",member.item_group)

In [256]:
member.item_group = np.where(member.item_group=="Drinks","Beverage",member.item_group)

In [257]:
member = member[member.item_group != "Tips"]

In [258]:
member.item_group = np.where(member.item_group.isin(['24 Hour Charity Event','24 Hour- Dinner/ Lunch Entry','24 Hour- Sponsorship','24 Hour- Ticket Sales']),"24 Hour of Tennis",member.item_group)

In [259]:
member.head()

,Unnamed: 0,trans_id,timestamp,name,item,item_group,status,quantity,subtotal,discount,tax,total,paid,refund,balance,acn,sys_id
0,0,7315,2015-01-03 11:00:00,Pia Lomax,Lessons with Joe Snailum (01/03/2015),Lessons,Paid,1.0,65.00,0.00,0.00,65.00,65.00,0.0,0.0,79,General Single Membership
1,1,37831,2015-08-12 21:30:00,Jennifer Hernandez,$1 Snacks,Snacks,Paid,1.0,0.92,0.00,0.08,1.00,1.00,0.0,0.0,566,General Family Membership
2,2,37844,2015-08-13 10:50:00,Douglas Draeger,Java Jolt,Beverage,Paid,1.0,5.31,0.53,0.39,5.17,5.17,0.0,0.0,1532,General Property Owner
3,3,37843,2015-08-13 10:50:00,Douglas Draeger,Java Jolt,Beverage,Paid,1.0,5.31,0.53,0.39,5.17,5.17,0.0,0.0,1532,General Property Owner
4,4,37842,2015-08-13 10:50:00,Douglas Draeger,Mango Tango,Beverage,Paid,1.0,4.39,0.44,0.33,4.28,4.28,0.0,0.0,1532,General Property Owner


In [260]:
df = member[["trans_id","timestamp","acn","item","item_group","total","sys_id"]]

In [261]:
df.to_csv("df_member.csv")

In [262]:
guest.item_group.unique()

array(['24 Hour Charity Event', '24 Hour- Sponsorship', 'Accessories',
       'Adult Clinics', 'Apparel', 'Beer', 'Camp',
       'Catie Johnson Home School', 'CDM- MISC', 'Date Night', 'Dinners',
       'Drinks', 'Facility Rental', 'Fitness Special Event',
       'Food & Beverage', 'Food Allowance', 'Gift Certificate Liability',
       'Guest Fee', 'Junior Clinics', 'Kid Fit', 'Leagues', 'Lessons',
       'Massage', 'Membership Dues', 'Membership Initiation ',
       'Naturally Cafe', 'Naturally Juices', 'Naturally Lunch Special',
       'Naturally- Sandwiches', 'Naturally-Salad', 'Naturally-Smoothies',
       'Package Liability', 'Personal Training', 'Racquet Rental',
       'Racquets', 'Red Wine', 'Restring/ Labor Only', 'Shoes',
       'Smoothie', 'Snacks', 'Social Event', 'Special Event',
       'Sponsorship', 'String', 'Swim Camp', 'Teen Performance Camp',
       'Tennis Balls', 'Tips', 'White Wine'], dtype=object)

---
# Concat Member and Guest

In [165]:
member = pd.read_csv("df_member.csv")
guest = pd.read_csv("df_guest.csv")

In [166]:
print(member.shape)
print(guest.shape)

(135822, 8)
(6124, 9)


In [167]:
member = member[['trans_id', 'timestamp', 'acn', 'item', 'item_group','total', 'sys_id']]
guest = guest[['trans_id', 'timestamp', 'acn', 'item','item_group', 'total', 'sys_id']]

In [168]:
member.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135822 entries, 0 to 135821
Data columns (total 7 columns):
trans_id      135822 non-null int64
timestamp     135822 non-null object
acn           135822 non-null int64
item          135822 non-null object
item_group    135822 non-null object
total         135822 non-null float64
sys_id        135822 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 7.3+ MB


In [169]:
guest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6124 entries, 0 to 6123
Data columns (total 7 columns):
trans_id      6124 non-null int64
timestamp     6124 non-null object
acn           6124 non-null int64
item          6124 non-null object
item_group    6124 non-null object
total         6124 non-null float64
sys_id        6124 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 335.0+ KB


In [170]:
df = pd.concat([member,guest],ignore_index=True)

In [171]:
df.item.dtype

dtype('O')

In [172]:
for col in df:
    if df[col].dtype == "object":
        df[col].str.strip()

In [173]:
df.head()

,trans_id,timestamp,acn,item,item_group,total,sys_id
0,7315,2015-01-03 11:00:00,79,Lessons with Joe Snailum (01/03/2015),Lessons,65.00,General Single Membership
1,37831,2015-08-12 21:30:00,566,$1 Snacks,Snacks,1.00,General Family Membership
2,37844,2015-08-13 10:50:00,1532,Java Jolt,Beverage,5.17,General Property Owner
3,37843,2015-08-13 10:50:00,1532,Java Jolt,Beverage,5.17,General Property Owner
4,37842,2015-08-13 10:50:00,1532,Mango Tango,Beverage,4.28,General Property Owner


In [182]:
sorted(list(df.item_group.unique()))

['24 Hour of Tennis',
 'Accessories',
 'Adult Clinics',
 'Apparel',
 'Beer',
 'Beverage',
 'Camp',
 'Court Time',
 'Date Night',
 'Dinners',
 'Facility Rental',
 'Fitness Lessons',
 'Fitness Special Event',
 'Food & Consumables',
 'Food Allowance',
 'Guest Fee',
 'Homeowner Membership Initiation',
 'Junior Clinics',
 'Kid Fit',
 'Leagues',
 'Lessons',
 'Membership Dues',
 'Membership Initiation ',
 'Personal Training',
 'Racquet Rental',
 'Racquets',
 'Restring/ Labor Only',
 'Shoes',
 'Snacks',
 'Social Event',
 'Special Event',
 'Sponsorship',
 'String',
 'Swim Camp',
 'Swim Lessons ',
 'Teen Performance Camp',
 'Tennis Balls',
 'Tennis Lessons',
 'Wine']

In [175]:
#Soccer -MSOS
#Credits

In [179]:
df = df[df.item_group!='Soccer- MSOS']
df = df[df.item_group!= "Credits"]

In [183]:
df.to_csv("all_data.csv")